In [1]:
from vectorization.semantic_vector_database import SemanticVectorDatabase
from llms.providers import LLMRecommender
from generation.generator_simple import CodeSenseGenerator
from rag.rag import BasicContentRag

# Test requests
test_requests = [
    "I want to add a new rule to calculate loyalty points",
    "Create a new loyalty point reward for VIP customers",
    "Add validation for minimum order amount in loyalty rules",
    "Implement a service to check customer loyalty status",
    "Add integration with payment service for loyalty points"
]

generation_results = "results/generation"
def run_generator(vector: SemanticVectorDatabase, embedding_model: str, collection_name: str, recommender: LLMRecommender):
    """Demo function to test implementation generation"""

    collection = vector.get_collection(collection_name)
    rag = BasicContentRag(collection)

    generator = CodeSenseGenerator(rag, recommender)

    for sequence, request in enumerate(test_requests):
        print(f"\n{'='*80}")
        print(f"TESTING REQUEST: {request}")
        print(f"{'='*80}")

        try:
            # Generate suggestion
            coding_advice = generator.fetch_coding_advice(
                user_request=request
            )

            # Display results
            coding_advice.display()

            # Save suggestion
            coding_advice.save(f"{generation_results}/{embedding_model}/{collection_name}/test{sequence}.json")

        except Exception as e:
            print(f"Error testing {recommender}: {e}")

In [2]:
import os
from dotenv import load_dotenv
from llms.anthropic import AnthropicRecommender
from llms.ollama import OllamaRecommender
from llms.openai import OpenAIRecommender
from vectorization.semantic_vector_database import SemanticVectorDatabase

load_dotenv()

vector_results = "results/vectorization"
rag_results = "results/rag"

chroma_base = "{model}/loyalty_points_kb"

loyalty_collection = "loyalty_code_semantics_{llm}"
anthropic = AnthropicRecommender(os.getenv("ANTHROPIC_API_KEY"))
ollama = OllamaRecommender()
openai = OpenAIRecommender(os.getenv("OPENAI_API_KEY"))

models = [
    {
        "model": "all-MiniLM-L6-v2",
        "llms": ["claude3.5", "codellama", "gpt4.1"],
        "recommenders": [anthropic, ollama, openai]
    },
    {
        "model": "all-mpnet-base-v2",
        "llms": ["claude3.5", "codellama", "gpt4.1"],
        "recommenders": [anthropic, ollama, openai]
    }
]

rag_testers = []
reports = []

for model in models:
    embedding_model = model["model"]
    print("embedding_model: ", embedding_model)
    db_path = f"{vector_results}/{chroma_base.format(model=embedding_model)}"
    print("db_path: ", db_path)

    for i, llm in enumerate(model["llms"]):
        collection_name = loyalty_collection.format(llm=llm)
        recommenders = model["recommenders"][i]

        vector_db = SemanticVectorDatabase(db_path, embedding_model)
        run_generator(vector_db, embedding_model, collection_name, recommenders)


Initialized Anthropic provider with model: claude-3-5-sonnet-20241022
Initialized Ollama provider with model: codellama:7b
Initialized OpenAI provider with model: gpt-4.1
embedding_model:  all-MiniLM-L6-v2
db_path:  results/vectorization/all-MiniLM-L6-v2/loyalty_points_kb


D:\src\ai-agents\code-navigator-agent\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Initialized Chroma database at: results\vectorization\all-MiniLM-L6-v2\loyalty_points_kb
Using embedding model: all-MiniLM-L6-v2

TESTING REQUEST: I want to add a new rule to calculate loyalty points

=== Generating Implementation Suggestion ===
Request: I want to add a new rule to calculate loyalty points
Provider: Anthropic-claude-3-5-sonnet-20241022

1. Retrieving business context via RAG...

=== Performing Semantic Search ===
Query: 'I want to add a new rule to calculate loyalty points'
Retrieving top 3 results...

Found 3 results:

2. Generating implementation with Anthropic-claude-3-5-sonnet-20241022...

--- Result 1 (Distance: 0.9311) ---
File: D:\src\learning\dotnet\event-driven-course\module5\src\PlantBasedPizza.LoyaltyPoints\application\PlantBasedPizza.LoyaltyPoints.Internal\Services\LoyaltyService.cs
Business Purpose: Manages customer loyalty points for a plant-based pizza business, allowing customers to earn and track points based on their orders
Technical Pattern: Microser